In [1]:
from selenium import webdriver
from selenium_stealth import stealth
from selenium.webdriver.common.by import By

def open_stealth_driver(headless=False, maximize=True, options=None):
    if options is None:
        options = webdriver.ChromeOptions()
        if maximize:
            options.add_argument("start-maximized")
        if headless:
            options.add_argument("--headless")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(options=options)
    stealth(driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
            )
    return driver

In [136]:
import pandas as pd
import numpy as np
import os
import json

In [137]:
# arguments
data_folder_path = '../public/data'
collections = {
"genesis": ["CEA-2d29f9", "GSPACEAPE-08bc2b"],
"heroes": ["CRHEROES-9edff2"],
"weapons": ["CRWEAPONS-e5ab49", "CRMYTH-546419"],
}
# load data
with open(os.path.join(data_folder_path, 'characters_upgrade.json')) as f:
    characters_upgrade = json.load(f)
with open(os.path.join(data_folder_path, 'weapons_upgrade.json')) as f:
    weapons_upgrade = json.load(f)
genesis = pd.concat([pd.read_json(os.path.join(data_folder_path, collection, 'nfts.json'), orient='index') for collection in collections['genesis']])
heroes = pd.concat([pd.read_json(os.path.join(data_folder_path, collection, 'nfts.json'), orient='index') for collection in collections['heroes']])
weapons = pd.concat([pd.read_json(os.path.join(data_folder_path, collection, 'nfts.json'), orient='index') for collection in collections['weapons']])
# get CRT-EGLD rate
stealth_driver = open_stealth_driver(headless=True)
url = 'https://coindataflow.com/en/pair/crt-wegld'
stealth_driver.get(url)
crt_egld_rate = float(stealth_driver.find_element(By.ID, 'converter').get_attribute('data-rate'))
stealth_driver.close()
market_data = {'CRT/EGLD': crt_egld_rate, 'floorPrice': {'genesis': {}, 'heroes': {}, 'weapons': {}}}

In [138]:
def get_character_value(floor_price, level, tokens):
    shard_conversion = 1
    token_conversion = 400
    shards = 0
    crown = 0
    for l in range(1, level+1):
        shards += characters_upgrade['nft'][str(l)]['shards']
        tokens += characters_upgrade['nft'][str(l)]['tokens']
        crown += characters_upgrade['nft'][str(l)]['crown']
    shards = shards * shard_conversion /100 * crt_egld_rate
    crown = crown /100 * crt_egld_rate
    tokens = tokens * token_conversion /100 * crt_egld_rate
    return floor_price + shards + tokens + crown

def get_character_floorPrice(df):
    rarities = df.value_counts('rarityClass').reset_index()
    rarities['percent'] = rarities['count'] / rarities['count'].sum()
    onsale = df[(~df['priceAmount'].isna()) & (df['priceCurrency']=='EGLD')]
    rarities = pd.merge(rarities, onsale.groupby('rarityClass').agg({'priceAmount':'min'}).reset_index().rename(columns={'priceAmount':'floorPrice'}), on='rarityClass', how='outer')
    rarities = rarities.fillna(0)
    rarities_dict = rarities.to_dict(orient='index')
    while True:
        last = -1
        for index in range(len(rarities_dict)):
            if rarities_dict[index]['floorPrice'] > 0:
                last = index
            if last != -1 and rarities_dict[index]['floorPrice'] == 0:
                rarities_dict[index]['floorPrice'] = rarities_dict[last]['percent']/rarities_dict[index]['percent'] * rarities_dict[last]['floorPrice']
        last = -1
        for index in range(len(rarities_dict)-1, -1, -1):
            if rarities_dict[index]['floorPrice'] > 0:
                last = index
            if last != -1 and rarities_dict[index]['floorPrice'] == 0:
                rarities_dict[index]['floorPrice'] = rarities_dict[last]['percent']/rarities_dict[index]['percent'] * rarities_dict[last]['floorPrice']
        rarities = pd.DataFrame(rarities_dict).T
        if rarities[rarities['floorPrice']==0].shape[0] > 0:
            rarities = rarities.sort_values('percent', ascending=False)
            rarities.iloc[0, rarities.columns.get_loc('floorPrice')] = 5
            rarities_dict = rarities.to_dict(orient='index')
        else:
            break
    rarities = rarities.set_index('rarityClass')
    rarities = rarities.to_dict(orient='index')
    return rarities

In [139]:
for name, df in [('genesis', genesis), ('heroes', heroes)]:
    floorPrice = get_character_floorPrice(df)
    market_data['floorPrice'][name] = floorPrice
    df['EGLDvalue'] = df.apply(lambda x: get_character_value(floorPrice[x['rarityClass']]['floorPrice'], x['level'], x['characterTokens']), axis=1)
    df['flag'] = (~df['priceAmount'].isna()) & (df['priceCurrency']=='EGLD')
    df['discount'] = df.apply(lambda x: x['priceAmount'] - x['EGLDvalue'] if x['flag'] else np.nan, axis=1)
    df['discount'] = df.apply(lambda x: x['discount'] / x['EGLDvalue'] * 100 if x['flag'] else np.nan, axis=1)
    df['discount'] = df['discount'].apply(lambda x: x if x < 0 else np.nan)
    df = df.drop(columns=['flag'])
    df = df.sort_values('discount', ascending=True)
    for collection in df['collection'].unique():
        collection_df = df[df['collection']==collection]
        collection_df.to_json(os.path.join(data_folder_path, collection, 'nfts.json'), orient='index', indent=4)

In [147]:
def get_weapon_value(floor_price, starLevel, level, tokens):
    shards_fusion = {
        1: 0,
        2: 1000,
        3: 6000,
        4: 35000,
        5: 100000,
        6: 500000
    }
    shard_conversion = 1
    token_conversion = 400
    shards = 0
    crown = 0
    for l in range(1, starLevel+1):
        shards += shards_fusion[l]
    for l in range(1, level+1):
        shards += weapons_upgrade['nft'][str(l)]['shards']
        tokens += weapons_upgrade['nft'][str(l)]['tokens']
        crown += weapons_upgrade['nft'][str(l)]['crown']
    shards = shards * shard_conversion /100 * crt_egld_rate
    crown = crown /100 * crt_egld_rate
    tokens = tokens * token_conversion /100 * crt_egld_rate
    return floor_price + shards + tokens + crown


def get_weapon_floorPrice(df):
    def foo(starlevel):
        starlevel = int(starlevel)
        result = 1
        for i in range(1, starlevel):
            result *= 3
        return result
    df = df[~df['starLevel'].isna()]
    df['countStarLevel1'] = df['starLevel'].apply(foo)
    rarities = df.groupby('name').agg({'countStarLevel1':'sum'}).reset_index()
    onsale = df[(~df['priceAmount'].isna()) & (df['priceCurrency']=='EGLD')]
    onsale['priceStarlevel1'] = onsale['priceAmount'] / onsale['countStarLevel1']
    tmp = onsale.groupby('name').agg({'priceStarlevel1':'min'}).reset_index().rename(columns={'priceStarlevel1':'floorPrice'})
    rarities = pd.merge(rarities, tmp, on='name', how='outer')
    rarities['countStarLevel1%'] = rarities['countStarLevel1'] / rarities['countStarLevel1'].sum()
    rarities = rarities.sort_values('countStarLevel1%', ascending=False).reset_index(drop=True)
    rarities = rarities.fillna(0)
    rarities = rarities.rename(columns={'countStarLevel1':'count1*', 'countStarLevel1%':'percent1*'})
    rarities_dict = rarities.to_dict(orient='index')
    while True:
        last = -1
        for index in range(len(rarities_dict)):
            if rarities_dict[index]['floorPrice'] > 0:
                last = index
            if last != -1 and rarities_dict[index]['floorPrice'] == 0:
                rarities_dict[index]['floorPrice'] = rarities_dict[last]['percent1*']/rarities_dict[index]['percent1*'] * rarities_dict[last]['floorPrice']
        last = -1
        for index in range(len(rarities_dict)-1, -1, -1):
            if rarities_dict[index]['floorPrice'] > 0:
                last = index
            if last != -1 and rarities_dict[index]['floorPrice'] == 0:
                rarities_dict[index]['floorPrice'] = rarities_dict[last]['percent1*']/rarities_dict[index]['percent1*'] * rarities_dict[last]['floorPrice']
        rarities = pd.DataFrame(rarities_dict).T
        if rarities[rarities['floorPrice']==0].shape[0] > 0:
            rarities = rarities.sort_values('percent1*', ascending=False)
            x = 1 if df['collection'].unique()[0] == 'CRMYTH-546419' else 0.1
            rarities.iloc[0, rarities.columns.get_loc('floorPrice')] = x
            rarities_dict = rarities.to_dict(orient='index')
        else:
            break
    rarities = rarities.set_index('name')
    rarities = rarities.to_dict(orient='index')
    return rarities

In [148]:
for collection in weapons['collection'].unique():
    df = weapons[weapons['collection']==collection]
    floorPrice = get_weapon_floorPrice(df)
    market_data['floorPrice']['weapons'].update(floorPrice)

/tmp/ipykernel_334946/3721410849.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['countStarLevel1'] = df['starLevel'].apply(foo)


{'Tesla Gun X': {'count1*': 17148,
  'floorPrice': 0.1,
  'percent1*': 0.22073759413014096},
 'Boomstick-X': {'count1*': 16947,
  'floorPrice': 0.10118605062842981,
  'percent1*': 0.21815022205058893},
 'Blaster-X': {'count1*': 16223,
  'floorPrice': 0.1057017814214387,
  'percent1*': 0.2088305335650383},
 'Mortar Gun X': {'count1*': 15126,
  'floorPrice': 0.11336771122570409,
  'percent1*': 0.19470940335972195},
 'Railgun-X': {'count1*': 12241,
  'floorPrice': 0.14008659423249734,
  'percent1*': 0.15757224689450988}}